In [ ]:
import os
os.environ["THEANO_FLAGS"] = "device=gpu1"
os.environ["CPATH"] = ""
from lasagnekit.generative.autoencoder import Autoencoder, greedy_learn_with_validation

from lasagnekit.easy import BatchOptimizer, LightweightModel
from lasagnekit.datasets.mnist import MNIST

from sklearn.utils import shuffle


from sklearn.cross_validation import train_test_split

from lasagne import layers, updates, init, nonlinearities
import theano.tensor as T
from theano.sandbox import rng_mrg
import theano
import matplotlib.pyplot as plt
import numpy as np
from lasagne.layers import get_all_layers

from skimage.io import imread
from skimage.filter import threshold_otsu
from skimage.transform import resize

import matplotlib.pyplot as plt

from lasagnekit import easy

from lasagnekit.generative.capsule import Capsule
from lasagnekit.easy import BatchIterator
import glob
import os

from lasagnekit.easy import BatchOptimizer, LightweightModel
from lasagne import init
from collections import OrderedDict
from lasagne import init, layers, updates, nonlinearities
from lasagne.layers.helper import get_all_layers
from lasagne.layers import helper
import theano.tensor as T
from theano.sandbox import rng_mrg
from sklearn.cross_validation import train_test_split
from lasagnekit.datasets.fonts import Fonts
import theano
from collections import OrderedDict
import theano.tensor as T

from lasagnekit.generative.capsule import Capsule
from lasagne.layers import Layer
from lasagnekit.datasets.cached import Cached
from sklearn.preprocessing import label_binarize
from skimage.filter import threshold_otsu
from skimage.transform import resize
from lasagne.layers import cuda_convnet
#from lasagne.layers.dnn import Conv2DDNNLayer
from lasagne.layers import Conv2DLayer
from lasagnekit.datasets.rescaled import Rescaled
%matplotlib inline


def binarize(X):
    X_b = np.empty(X.shape, dtype=X.dtype)
    for i in range(X.shape[0]):
        X_b[i] = 1. * (X[i] <= threshold_otsu(X[i]))
    return X_b

def resize_all(X, w, h):
    if X.shape[1] == w and X.shape[2] == h:
        return X
    X_b = np.empty((X.shape[0], w, h), dtype=X.dtype)
    for i in range(X.shape[0]):
        X_b[i] = resize(X[i], (w, h))
    return X_b  

class SumLayer(Layer):
    def __init__(self, 
                 incoming,
                 axis=1,
                 **kwargs):
        super(SumLayer, self).__init__(incoming, **kwargs)
        self.axis = axis

    def get_output_for(self, input, **kwargs):
        return input.sum(axis=self.axis)
    
    def get_output_shape_for(self, input):
        shape = list(self.input_shape)
        del shape[self.axis]
        return tuple(shape)

class Model:
    def get_all_params(self, **t):
        return list(set(self.x_to_z.get_all_params(**t) + 
                        self.x_to_y.get_all_params(**t) + 
                        self.z_to_x.get_all_params(**t)))

def cross_correlation(a, b):
    a = a - a.mean(axis=0)
    b = b - b.mean(axis=0)
    return 0.5 * ((((a.dimshuffle(0, 'x', 1) * b.dimshuffle(0, 1, 'x'))).mean(axis=0))**2).sum()

%matplotlib inline

# Seed

In [ ]:
seed = 1234
np.random.seed(seed)

# Load & pre-process data

In [ ]:
dataset = "fonts"
w, h = 28, 28 # Desired resolution (if necessary the images will be resize to fit w and h)

In [ ]:

if dataset == "mnist":
    data = MNIST()
elif dataset == "fonts":
    data = Fonts(kind="all_64", 
                 labels_kind="letters")

data = Cached(Rescaled(data, (w, h)))
data.load()

X = data.X
y = data.y
output_dim = data.output_dim
real_w, real_h = data.img_dim
    
y = label_binarize(y, np.arange(output_dim))
y = y.astype(np.float32)

In [ ]:
print(X.shape, y.shape, output_dim)

In [ ]:
if dataset == "fonts":
    pass
    #filter_by_letter = (y[:, 19]==1) | (y[:, 20]==1)
    #X = X[filter_by_letter]
    #y = y[filter_by_letter]

In [ ]:
print(X.shape)
plt.imshow(X[20].reshape((w, h)), cmap="gray")
print(X.shape)

In [ ]:
X, y = shuffle(X, y)
train, test = train_test_split(range(X.shape[0]), test_size=0.25)

In [ ]:

nb_samples_for_learning_curve = 100
nb_tries_learning_curve = 10

class MyBatchOptimizer(BatchOptimizer):
    
    def iter_update(self, epoch, nb_batches, iter_update_batch):
        status = super(MyBatchOptimizer, self).iter_update(epoch, nb_batches, iter_update_batch)
        
        all_of_them = [
            "acc_train",
            "acc_valid",
            "rec_train",
            "rec_valid",
            "crosscor_train",
            "crosscor_valid"
        ]
        for a in all_of_them:
            status[a] = 0.
        for i in range(nb_tries_learning_curve):
        
            s = np.arange(len(train))
            np.random.shuffle(s)
            s_train = s[0:nb_samples_for_learning_curve]


            s = np.arange(len(test))
            np.random.shuffle(s)
            s_test = s[0:nb_samples_for_learning_curve]


            status["acc_train"] += (self.model.predict(X[train][s_train])==y[train][s_train].argmax(axis=1)).mean()
            status["acc_valid"] += (self.model.predict(X[test][s_test])==y[test][s_test].argmax(axis=1)).mean()

            status["rec_train"] += self.model.get_reconstruction_error(X[train][s_train])
            status["rec_valid"] += self.model.get_reconstruction_error(X[test][s_test])

            status["crosscor_train"] += self.model.get_cross_correlation(X[train][s_train])
            status["crosscor_valid"] += self.model.get_cross_correlation(X[test][s_test])
        for a in all_of_them:
            status[a] /= nb_tries_learning_curve
        return status

## Model definition

## model type

In [ ]:
model_type = "convnet" # or "convnet"

### Fully connected

In [ ]:
if model_type == "fully_connected":
    ## fully connected
    latent_size = 4
    num_hidden_units = 1000

    l_in = layers.InputLayer((None, w*h))
    input_dim = w*h
    output_dim = y.shape[1]

    # encoder
    l_encoder1 = layers.DenseLayer(l_in, num_units=num_hidden_units)
    l_encoder2 = layers.DenseLayer(l_encoder1, num_units=num_hidden_units)
    l_encoder3 = layers.DenseLayer(l_encoder2, num_units=num_hidden_units)
    l_encoder4 = layers.DenseLayer(l_encoder3, num_units=num_hidden_units)
    l_encoder4 = layers.DenseLayer(l_encoder4, num_units=num_hidden_units)


    # learned representation
    l_observed = layers.DenseLayer(l_encoder4, num_units=output_dim,
                                      nonlinearity=T.nnet.softmax)

    l_latent = layers.DenseLayer(l_encoder4, 
                                 num_units=latent_size,
                                 nonlinearity=None) # linear

    l_representation = layers.concat([l_observed, l_latent])

    # decoder
    l_decoder1 = layers.DenseLayer(l_representation, num_units=num_hidden_units)
    l_decoder2 = layers.DenseLayer(l_decoder1, num_units=num_hidden_units)
    l_decoder3 = layers.DenseLayer(l_decoder2, num_units=num_hidden_units)
    l_decoder4 = layers.DenseLayer(l_decoder3, num_units=num_hidden_units)
    l_encoder4 = layers.DenseLayer(l_encoder4, num_units=num_hidden_units)

    l_decoder_out = layers.DenseLayer(l_decoder4, num_units=input_dim,
                                       nonlinearity=nonlinearities.sigmoid)

    x_to_z = LightweightModel([l_in], [l_latent])
    x_to_y = LightweightModel([l_in], [l_observed])
    z_to_x = LightweightModel([l_observed, l_latent], [l_decoder_out])
    model = Model()
    model.x_to_z = x_to_z
    model.x_to_y = x_to_y
    model.z_to_x = z_to_x

### Convnet

In [ ]:
if model_type == "convnet":
    Layers = dict()
    deconv = True
    ## CNN
    latent_size = 10
    nb_filters=16
    size_filters=5
    nb_hidden=1000
    nb_filters_encoder = nb_filters
    nb_filters_decoder = nb_filters
    size_filters_encoder = size_filters
    size_filters_decoder = size_filters

    l_in = layers.InputLayer((None, w*h))


    x_in_reshaped = layers.ReshapeLayer(l_in, ([0], 1, w, h))

    # conv1
    l_conv = layers.Conv2DLayer(
        x_in_reshaped,
        num_filters=nb_filters_encoder,
        filter_size=(size_filters_encoder, size_filters_encoder),
        nonlinearity=nonlinearities.rectify,
        name="conv1",
    )
       
    l_hid = layers.DenseLayer(
        l_conv,
        num_units=nb_hidden,
        nonlinearity=nonlinearities.rectify,
        name="hid1"
    )
    
    l_hid = layers.DenseLayer(
        l_hid,
        num_units=nb_hidden,
        nonlinearity=nonlinearities.rectify,
        name="pre_code_layer"
    )
    l_hid = layers.DenseLayer(
        l_hid,
        num_units=nb_hidden,
        nonlinearity=nonlinearities.rectify,
        name="pre_code_layer"
    )
        

    #code layer

    l_observed = layers.DenseLayer(l_hid, 
                                   num_units=output_dim,
                                   nonlinearity=T.nnet.softmax,
                                   name="observed")

    l_latent = layers.DenseLayer(l_hid, 
                                 num_units=latent_size,
                                 nonlinearity=None,
                                 name="factors") # linear

    hid = layers.ConcatLayer([l_latent, l_observed], axis=1, name="concat_layer")

    l_hid = layers.DenseLayer(
        hid,
        num_units=nb_hidden,
        nonlinearity=nonlinearities.rectify,
        name="hid3"
    )
    
    l_hid = layers.DenseLayer(
        l_hid,
        num_units=nb_hidden,
        nonlinearity=nonlinearities.rectify,
        name="hid4"
    )
    
        
 
    if deconv is True:

        # unflatten layer
        hid = layers.DenseLayer(l_hid,
                                num_units=nb_filters_decoder * (w + 1*(size_filters_decoder - 1)) * 
                                (h + 1*(size_filters_decoder - 1)),
                                nonlinearity=nonlinearities.linear,
                                name="unflatten")
        hid = layers.ReshapeLayer(hid,
                                  ([0], nb_filters_decoder, 
                                   (w + 1*(size_filters_decoder - 1)), 
                                   (h + 1*(size_filters_decoder - 1))))

        l_unconv = Conv2DLayer(
            hid,
            num_filters=nb_filters,
            filter_size=(size_filters_decoder, size_filters_decoder),
            nonlinearity=nonlinearities.linear,            
            name="unconv1"
        )
        #l_unconv = Conv2DLayer(
        #    l_unconv,
        #    num_filters=nb_filters,
        #    filter_size=(size_filters_decoder, size_filters_decoder),
        #    nonlinearity=nonlinearities.linear,
        #    pad="full",
        #    name="unconv2"
        #)

        l_unconv_sum = SumLayer(l_unconv, axis=1, name="sum")
        l_unconv_sum = layers.ReshapeLayer(l_unconv_sum, ([0], 1, [1], [2]))
        #l_decoder_out = Conv2DLayer(
        #    l_unconv_sum,
        #    num_filters=1,
        #    filter_size=(3, 3),
        #    nonlinearity=nonlinearities.linear,
        #    pad="same",
        #    name="smooth"
        #)
        l_decoder_out = layers.ReshapeLayer(l_unconv_sum, ([0], w*h))
        l_decoder_out = layers.NonlinearityLayer(l_decoder_out, nonlinearities.sigmoid, name="reconstruction")
        

    else:
        l_decoder_out = layers.DenseLayer(l_hid, 
                                          num_units=w*h, 
                                          nonlinearity=nonlinearities.sigmoid, name="reconstruction")

        

    x_to_z = LightweightModel([l_in], [l_latent])
    x_to_y = LightweightModel([l_in], [l_observed])
    z_to_x = LightweightModel([l_observed, l_latent], [l_decoder_out])
    model = Model()
    model.x_to_z = x_to_z
    model.x_to_y = x_to_y
    model.z_to_x = z_to_x

## Visualization of the model

In [ ]:
from lasagnekit.misc.draw_net import draw_to_file
draw_to_file(get_all_layers(l_decoder_out), "model.svg")

from IPython.display import SVG
SVG("model.svg")

In [ ]:
layers_by_name = OrderedDict()


all_layers = (
    layers.get_all_layers(model.x_to_z.output_layers[0]) +
    layers.get_all_layers(model.x_to_y.output_layers[0]) +
    layers.get_all_layers(model.z_to_x.output_layers[0])
)

for l in all_layers:
    if l.name is not None:
        layers_by_name[l.name] = l
print(layers_by_name.keys())

## Define the capsule object

The Capsule object combines all the components:

    - The model
    - The training algorithm
    - The loss function

In [ ]:

# Define the Capsule object, which combines the Model/Training procedure/Loss function

input_variables = OrderedDict()
input_variables["X"] = dict(tensor_type=T.matrix)
input_variables["y"] = dict(tensor_type=T.matrix)
    

functions = dict(
    encode=dict(
        get_output=lambda model, X:model.x_to_z.get_output(X)[0],
        params=["X"]
    ),
    predict=dict(
        get_output=lambda model, X:(model.x_to_y.get_output(X)[0]).argmax(axis=1),
        params=["X"]
    ),
    reconstruct=dict(
        get_output=lambda model, X: layers.get_output(l_decoder_out, X),
        params=["X"]
    ),
    get_reconstruction_error=dict(
        get_output=lambda model, X: ((X-layers.get_output(l_decoder_out, X))**2).sum(axis=1).mean(),
        params=["X"]
    ),
    get_cross_correlation=dict(
        get_output=lambda model, X: cross_correlation(model.x_to_z.get_output(X)[0],
                                                      model.x_to_y.get_output(X)[0]),
        params=["X"]
    ),
    predict_proba=dict(
        get_output=lambda model, X: model.x_to_y.get_output(X)[0],
        params=["X"]
    ),
    
    get_all_layers=dict(
        get_output=lambda model, X: [layers.get_output(l, X) for l in layers_by_name.values()],
        params=["X"]
    ),
    
    #get_all_grads=dict(
    #    get_output=lambda model, X, y: [theano.grad(loss_function(model, dict(X=X, y=y)), 
    #                                                p) for p in model.get_all_params()],
    #    params=["X", "y"]
    #)


)

batch_optimizer = MyBatchOptimizer(
    verbose=1,
    max_nb_epochs=100,
    batch_size=100,
    optimization_procedure=(updates.rmsprop, 
                            {"learning_rate": 0.0001})
)



def loss_function(model, tensors):
    x_to_z, x_to_y, z_to_x = model.x_to_z, model.x_to_y, model.z_to_x
    X_batch, y_batch = tensors["X"], tensors["y"]
    
    z, = x_to_z.get_output(X_batch)
    
    
    pre_code = layers.get_output(layers_by_name["pre_code_layer"], X_batch)
    
    y_hat, = x_to_y.get_output(X_batch)
    X_hat, = z_to_x.get_output(y_hat, z)
    
    loss_rec = ((X_hat - X_batch) ** 2).sum(axis=1).mean()
    loss_supervised = ((y_hat - y_batch)**2).sum(axis=1).mean()
    

 
    return  (loss_rec + 
             10*loss_supervised + 
             15 * cross_correlation(z, y_hat))
    
capsule = Capsule(
    input_variables, 
    model,
    loss_function,
    functions=functions,
    batch_optimizer=batch_optimizer
)

Z_batch = T.matrix("z_batch")
capsule.decode = theano.function([Z_batch, capsule.v_tensors["y"]], 
                                  layers.get_output(l_decoder_out,{l_latent: Z_batch, 
                                                    l_observed: capsule.v_tensors["y"]}))


In [ ]:
capsule._build_functions()

## Training 

In [ ]:
capsule.fit(X=X[train], y=y[train])

## Visualization of features

In [ ]:
from lasagnekit.misc.plot_weights import grid_plot

if model_type == "convnet":
    layers_enc = get_all_layers(model.x_to_z.output_layers[0])
    layers_dec = get_all_layers(model.z_to_x.output_layers[0])
    for l in layers_enc[2], layers_dec[-4]:
        plt.clf()
        W = l.W.get_value()[:, 0]
        grid_plot(W, imshow_options={"cmap": "gray"})
        plt.show()
elif model_type == "fully_connected":
    layers_ = get_all_layers(l_decoder_out)
    for W in (layers_[1].W.get_value().T, layers_[-1].W.get_value()):
        plt.clf()
        print(W.shape)
        W = W.reshape((W.shape[0], w, h))
        grid_plot(W, imshow_options={"cmap": "gray"}, nbrows=10, nbcols=10)
        plt.show()

## Statistics

In [ ]:
from lasagnekit.easy import get_stat

layer = 0
train_stat = get_stat("rec_train", capsule.batch_optimizer.stats)
test_stat = get_stat("rec_valid", capsule.batch_optimizer.stats)
plt.yticks(np.arange(round(max(train_stat), 2), round(min(train_stat), 2), -1.5))
plt.plot(train_stat, label="train")
plt.plot(test_stat, label="valid")
plt.xlabel("epoch")
plt.ylabel("reconstruction error")
plt.legend()
plt.title("reconstruction")
plt.savefig("rec.png")
plt.show()

epoch = get_stat("epoch", capsule.batch_optimizer.stats)
acc_train = get_stat("acc_train", capsule.batch_optimizer.stats)
acc_valid = get_stat("acc_valid", capsule.batch_optimizer.stats)
plt.title("accuracy")
plt.yticks(np.arange(round(min(acc_train), 2), round(max(acc_train), 2), 0.02))
plt.plot(acc_train, label="train")
plt.plot(acc_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(loc='best')
plt.savefig("acc.png")
plt.show()

epoch = get_stat("epoch", capsule.batch_optimizer.stats)
acc_train = get_stat("crosscor_train", capsule.batch_optimizer.stats)
acc_valid = get_stat("crosscor_valid", capsule.batch_optimizer.stats)
plt.title("crosscor")
plt.yticks(np.arange(round(max(acc_train), 3), round(min(acc_train), 3), -0.005))
plt.plot(acc_train, label="train")
plt.plot(acc_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("crosscor")
plt.legend(loc='best')
plt.savefig("crosscor.png")
plt.show()

## Interactive sliders

In [ ]:

from IPython.html.widgets import (interact, interactive, 
                                  IntSliderWidget, IntSlider, FloatSliderWidget,
                                  ButtonWidget
                                  )
from IPython.display import display # Used to display widgets in the notebook

from IPython.html.widgets import *
from IPython.html import widgets


use_examples = False # init/work with examples for dataset or not
nb = 1000 # nb of examples to consider
max_nb_sliders = 5
T_ = test
x = X[T_][0:nb]


nb_outputs = y.shape[1]



z = capsule.encode(x)

if z.shape[1] > max_nb_sliders:
    params = np.random.choice(z.shape[1],
                              size=10)
else:
    params = np.arange(z.shape[1])

boundaries = OrderedDict()
for p in (params):
    boundaries["{0}".format(p)] = FloatSliderWidget(min=z[:, p].min(),
                                                    max=z[:, p].max(),
                                                    step=0.01,
                                                    value=0.)
d = 0
l = y[T_][d].argmax() 

def draw(**all_params):
    if use_examples is True:
        example = all_params["example"]
        del all_params["example"]
    label = all_params["label"]
    del all_params["label"]
    params = all_params
    
    if use_examples is True:
        z = capsule.encode(x[example:example + 1])
    else:
        z = np.zeros((1, latent_size), dtype="float32")
        z[0, :] = 0
    
    y_ = np.zeros(nb_outputs, dtype='float32')
    y_[label] = 1.
    y_ = y_[np.newaxis, :]
        
    for k, v in params.items():
        z[0][int(k)] = v
    plt.imshow(capsule.decode(z, y_)[0].reshape((w, h)), cmap="gray")
    

p = dict()
p.update(boundaries)

label_selector = IntSliderWidget(min=0,max=26,step=1,value=l)
p["label"] = label_selector

if use_examples is True:
    example_selector = IntSliderWidget(min=0,max=nb-1,step=1,value=d)
    p["example"] = example_selector

i = interact(**p)



def on_button_clicked(b):
    
    example = example_selector.get_state()["value"]
    z = capsule.encode(x[example:example + 1])
    for p in params:
        w = boundaries["{0}".format(p)]
        state = w.get_state()
        state["value"] = z[0, int(p)]
        w.set_state(state)
        w.send_state(state)
        
    state = label_selector.get_state()
    state["value"] = y[example].argmax()
    label_selector.set_state(state)
    label_selector.send_state(state)
    
    
draw_i = i(draw)

if use_examples is True:
    button = widgets.ButtonWidget(description="fit!")
    display(button)
    button.on_click(on_button_clicked)



# Sharpness

In [ ]:
def sharpness(images):
    _, gy, gx = np.gradient(images)
    return (np.sqrt(gx**2 + gy**2)).mean(axis=(1, 2)).mean()

nb = 10000
x_ = X[test][0:nb]
x_hat_ = capsule.reconstruct(x_)

x_ = x_.reshape((x_.shape[0], w, h))
x_hat_ = x_hat_.reshape((x_hat_.shape[0], w, h))

# lower = less sharp, more blurry
print("original", sharpness(x_))
print("reconstruction", sharpness(x_hat_))
print(sharpness(x_hat_)/sharpness(x_))

## Visualizaion of reconstruction of images

In [ ]:
nb = 100
x_ = X[test][0:nb]
x_hat_ = capsule.reconstruct(x_)

In [ ]:
for i in range(nb):
    plt.clf()
    plt.subplot(1, 2, 1)
    plt.imshow(x_[i].reshape((w, h)), cmap="gray")
    plt.subplot(1, 2, 2)
    plt.imshow(x_hat_[i].reshape((w, h)), cmap="gray")
    plt.show()

## 2d histogram of hidden factors

In [ ]:
# SOurce :http://oceanpython.org/2013/02/25/2d-histogram/
x_ = X[0:20000]
z = capsule.encode(x_)

# Estimate the 2D histogram
nbins = 200
H, xedges, yedges = np.histogram2d(z[:, 0],z[:, 1],bins=nbins)
 
# H needs to be rotated and flipped
H = np.rot90(H)
H = np.flipud(H)
 
# Mask zeros
Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
 
# Plot 2D histogram using pcolor
fig2 = plt.figure()
plt.pcolormesh(xedges,yedges,Hmasked, cmap="afmhot")
plt.xlabel('x')
plt.ylabel('y')
cbar = plt.colorbar()
cbar.ax.set_ylabel('Counts')

## Checking the gaussianity of the hidden factors

In [ ]:
from scipy.stats import kurtosistest
import numpy as np
x_ = X[0:10000]
z = capsule.encode(x_)

_, pvalues = kurtosistest(z)
print(pvalues)

## Covariance matrix of hidden factors

In [ ]:
x_ = X[0:5000]
z = capsule.encode(x_)
print(z.mean(axis=0))
plt.matshow(np.abs(np.corrcoef(z.T)))
plt.colorbar()
np.save("hidfact_corr{0}_{1}".format(latent_size, model_type), np.corrcoef(z.T))

## Latent order by variance

In [ ]:
x_ = X[0:5000]
z = capsule.encode(x_)

C = np.cov(z.T)
latent_order = np.argsort(np.diag(C))[::-1]
print(latent_order)

## Visualization of samples when varying hidden factors

In [ ]:
from lasagnekit.misc.plot_weights import grid_plot
nb = 10# nb of different hidden factors values to consider
labels = np.arange(output_dim)# labels to consider (by default, all)
labels = np.arange(10)
std_units=4# nb of std units of values of latent dim around the mean to consider
L = latent_size

x_ = X[train][0:100]
z_ = capsule.encode(x_)
latent_std = np.std(z_, axis=0)
latent_mean = np.mean(z_, axis=0)

k = 1
for latent_dim in latent_order:
    print("hidden factor : {0}".format(latent_dim))
    ys = np.eye(output_dim)[labels].repeat(nb, axis=0)
    seq = np.linspace(latent_mean[latent_dim] - latent_std[latent_dim]*std_units,
                      latent_mean[latent_dim] + latent_std[latent_dim]*std_units,
                      nb)
    z = np.zeros((nb, L))
    z[:, latent_dim] = seq
    z = z.repeat(len(labels), axis=0)
    z = z.reshape((nb, len(labels), L))
    z = z.transpose((1, 0, 2))
    z = z.reshape((nb*len(labels), L))
    z = z.astype(np.float32)
    ys = ys.astype(np.float32)

    c = capsule.decode(z, ys)
    c = c.reshape((c.shape[0], w, h))
    plt.clf()
    grid_plot(c, imshow_options=dict(cmap="gray"), nbrows=len(labels), nbcols=nb)
    plt.savefig("hidfactor{0}.png".format(k))
    plt.show()
    k += 1